
## NPC Hunter

We show how to use EZKL to prove that the NPC hunter updates are "optimal". Given coordinates x,y for the hunter and the prey, the hunter moves to the closest point in the grid to the prey. 

The hunter can move in the four cardinal directions, selected using `argmin` over the calculated distances. 


In [ ]:
! pip3 install -U ezkl 
import ezkl 
ezkl.__version__

In [40]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch

In [41]:
class NPC(nn.Module):
    def __init__(self):
        super(NPC, self).__init__()

    def forward(self, state):
        # state is a tensor of shape (4,) where the first two elements are the coordinates of the prey and the last two elements are the coordinates of the hunter
        # the prey is at (state[0], state[1]) and the hunter is at (state[2], state[3])
        # (if we move hunter by +1 in x direction what is the distance ?) 
        dist_x_plus = torch.abs(state[:, 0] - (state[:, 2] + 1)) + torch.abs(state[:, 1] - state[:, 3])
        # (if we move hunter by -1 in x direction what is the distance ?)
        dist_x_minus = torch.abs(state[:, 0] - (state[:, 2] - 1)) + torch.abs(state[:, 1] - state[:, 3])
        # (if we move hunter by +1 in y direction what is the distance ?)
        dist_y_plus = torch.abs(state[:, 0] - state[:, 2]) + torch.abs(state[:, 1] - (state[:, 3] + 1))
        # (if we move hunter by -1 in y direction what is the distance ?)
        dist_y_minus = torch.abs(state[:, 0] - state[:, 2]) + torch.abs(state[:, 1] - (state[:, 3] - 1))

        # argmin = 0 -> dist_x_plus
        # argmin = 1 -> dist_x_minus
        # argmin = 2 -> dist_y_plus
        # argmin = 3 -> dist_y_minus
        argmin = torch.argmin(torch.stack([dist_x_plus, dist_x_minus, dist_y_plus, dist_y_minus], dim=0), dim=0)
        return argmin

circuit = NPC()

In [62]:
gip_run_args = ezkl.PyRunArgs()
gip_run_args.input_visibility = "public"  # matrix and generalized inverse commitments
gip_run_args.output_visibility = "public"   
gip_run_args.param_visibility = "fixed" # should be Tensor(True)
gip_run_args.logrows = 15
gip_run_args.decomp_base = 2**6

In [64]:
# After training, export to onnx (network.onnx) and create a data file (input.json)
shape = [1, 4]

A = torch.randint(size=shape, low=0, high=10, dtype=torch.int64)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      (A),  # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'}},    # variable length axes
                      )

data = dict(
    input_data=A.tolist(),
)

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


In [65]:
circuit.forward(A)

tensor([0])

In [72]:


res = ezkl.gen_settings(py_run_args=gip_run_args)

assert res == True

res = await ezkl.calibrate_settings(data="input.json")




 <------------- Numerical Fidelity Report (input_scale: 0, param_scale: 13, scale_input_multiplier: 10) ------------->

+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error | median_error | max_error | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0          | 0            | 0         | 0         | 0              | 0                | 0             | 0             | 0                  | 0                  | 0                      |
+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+-

In [73]:
res = ezkl.compile_circuit()
assert res == True

In [74]:
# srs path
res = await ezkl.get_srs( settings_path)

In [75]:
# now generate the witness file

res = await ezkl.gen_witness()


In [76]:

# we pass the witness file to the setup function so as to prepopulate the "fixed" columns of the circuit. 
# in this case we want to force the output to be 0 meaning that the difference between the two matrices is 0
res = ezkl.setup(witness_path = "witness.json")

assert res == True

In [77]:
# GENERATE A PROOF



res = ezkl.prove(proof_path="proof.json")

print(res)


{'instances': [['0400000000000000000000000000000000000000000000000000000000000000', '0800000000000000000000000000000000000000000000000000000000000000', '0100000000000000000000000000000000000000000000000000000000000000', '0100000000000000000000000000000000000000000000000000000000000000', '0000000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x0f8ae90e591291eebabd442c2ade1245cc5d04a8d4f780370b2e7f08b0f040b604130ecd6387d9253548441060bb32591d843167554b2917cb709473bd5322a01261cc95767c2c10be2a72d080e47b5a4e6a1062e70c318b58aff501d62a4ac51775cd6c813db6db9d3e25273086db41873f577c4ec067414b5fcc48c300f8ee10d2b4364570a1527cd3ba2dbbe71e8f58b7a6b3bef7632e67621ce3b850971e21da2f0b55e7d3e645a81c4b94ec82a23b4d0bb87f2f9d6783b2405cf24c995308e4630557c37b4b5a4d2092d122b3e3fced6627da4724e431934e89cdabc9761ec2fee618300bdac7ee88dd19d88aac8a44b22837df1b028d69489f71bff17b286d7b46d04dea05e5b286c59de9b3f7bf5d89e6947381fe5950a8ae8ef91a1521242a1158cc560eabdf78d13f0bb12208d5d80b93c75bdb1bc59b

In [78]:
# VERIFY IT

res = ezkl.verify()

assert res == True
print("verified")

verified


We can now create and then deploy a vanilla evm verifier. Which can be used to verify the correctness of the hunter's moves ! 

In [79]:
abi_path = 'test.abi'
sol_code_path = 'test.sol'

res = await ezkl.create_evm_verifier(
        sol_code_path=sol_code_path,
        abi_path=abi_path
    )
assert res == True

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2763959909.py, line 5)